In [1]:
#Ramadan mubarak
import os
#nltk will be the main library here 
from nltk.tokenize import word_tokenize #self explanatory
from nltk.corpus import stopwords #self explanatory
from nltk.stem import WordNetLemmatizer #self explanatory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from transformers import pipeline

2024-03-23 23:00:45.710342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 23:00:45.710519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 23:00:45.879512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
generator = pipeline('text-generation', model ='EleutherAI/gpt-neo-1.3B')
prompt1 = "The industrial revolution and its consequences have been a disaster for the human race."
output1 = generator(prompt1, max_length=300, do_sample=True, temperature=0.9)
prompt2 = "Social media is a great inhouse threat for families that could result in long term harm."
output2 = generator(prompt2, max_length=350, do_sample=False, temperature=0.7)
prompt3 = "The fall of the great umayyad caliphate had negative effects on its territories."
output3 = generator(prompt3, max_length=450, do_sample=True, temperature=0.9)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [3]:
with open('industrial.txt', 'w') as f:
       f.write(str(output1))
with open('social.txt', 'w') as f:
       f.write(str(output2))
with open('umayyad.txt', 'w') as f:
       f.write(str(output3))

In [4]:
docs_path = "/kaggle/working"
docs = [] 

In [5]:
#task 1
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        with open(os.path.join(docs_path, filename), "r")  as file:
            document = file.read()
            docs.append(document)

In [6]:
class CustomTFIDF:
    def __init__(self):
        self.idf_ = {}
        self.vocab_ = {}

    def fit_transform(self, documents):
        tf = []
        doc_count = len(documents)

        # Compute term frequencies and document frequencies for IDF
        for document in documents:
            doc_tf = {}
            words = document.split()
            for word in words:
                doc_tf[word] = doc_tf.get(word, 0) + 1
            for word in doc_tf:
                doc_tf[word] = doc_tf[word] / len(words)
                self.idf_[word] = self.idf_.get(word, 0) + 1
            tf.append(doc_tf)

        # Sort the vocabulary alphabetically and assign indices
        sorted_vocab = sorted(self.idf_.keys())
        self.vocab_ = {word: idx for idx, word in enumerate(sorted_vocab)}

        # Compute IDF using the sorted vocabulary
        for word in self.idf_:
            self.idf_[word] = np.log((1 + doc_count) / (1 + self.idf_[word])) + 1

        # Compute TF-IDF scores using the sorted vocabulary
        tfidf = []
        for doc in tf:
            doc_tfidf = np.zeros(len(self.vocab_))
            for word, value in doc.items():
                if word in self.vocab_:
                    index = self.vocab_[word]
                    doc_tfidf[index] = value * self.idf_[word]
            # L2 Normalization
            norm = np.linalg.norm(doc_tfidf)
            if norm > 0:
                doc_tfidf = doc_tfidf / norm
            tfidf.append(doc_tfidf)

        return np.array(tfidf)

## preprocessing

In [7]:
#this is something about kaggle and nltk don't ask me
import nltk
import subprocess
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.

In [8]:
#task 2
all_unique_words = []
for document in docs:
    temp = []
    preprocessed_text = document.lower().strip() #lower case + remove spaces
    tokens = word_tokenize(preprocessed_text) #tokenization
    lemmatizes = [WordNetLemmatizer().lemmatize(token) for token in tokens] #lemmatization
    stop_words = stopwords.words('english')
    filtered_tokens = [token for token in lemmatizes if token.isalpha() and token not in stop_words and len(token) > 3] #removing stop words and isalpha for filtering non alphabetical 
    #unique_words.append(filtered_tokens) this failed because the tfidf can accept a list of strings not a list of lists
    temp.extend(filtered_tokens)
    doc_unique_words = set(temp)
    doc_unique_string = " ".join(doc_unique_words)
    all_unique_words.append(doc_unique_string)
print(all_unique_words)

['lived territory though control negative much small cairo fell changed baghdad century turn power like become time caliphate midst however umayyad leader centralized damascus greater muslim scattered great greatest capital place weak resulted contained tribal army authority greatly called caliph part unity political world series concentrated ruler amount empire sunni mountain heart year inside increased wall fall islamic split leadership effect resided generally capacity expand long among result little divided might longer nation unified confederacy seventh city base aleppo united situation people last fact rule', 'post threat inhouse adhd guest could term friend year family following medium parent seen great child diagnosed many long also result member', 'difficult knowledge need better important seems becoming paper natural confess disaster survival dimension reality point rather revolution upon blog human race idea reading possible environment read rely matter intelligent place enc

In [9]:
#task 3
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_unique_words)
features = tfidf_vectorizer.get_feature_names_out()

In [10]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
print(features)
print(tfidf_matrix.toarray())

['action' 'adhd' 'aleppo' 'also' 'among' 'amount' 'answer' 'arisen' 'army'
 'authority' 'baghdad' 'base' 'become' 'becoming' 'believe' 'benefit'
 'better' 'blog' 'cairo' 'caliph' 'caliphate' 'called' 'capacity'
 'capital' 'centralized' 'century' 'changed' 'child' 'city' 'concentrated'
 'confederacy' 'confess' 'consequence' 'contained' 'control' 'could'
 'damascus' 'diagnosed' 'difficult' 'dimension' 'disaster' 'discus'
 'discussion' 'divided' 'easy' 'effect' 'empire' 'encountered' 'entirely'
 'environment' 'expand' 'exploit' 'fact' 'fall' 'family' 'fell' 'find'
 'first' 'following' 'friend' 'future' 'generally' 'generation' 'good'
 'great' 'greater' 'greatest' 'greatly' 'guest' 'harness' 'heart'
 'however' 'human' 'idea' 'important' 'increased' 'industrial' 'inhouse'
 'inside' 'intelligent' 'islamic' 'knowledge' 'last' 'leader' 'leadership'
 'like' 'likely' 'little' 'live' 'lived' 'long' 'longer' 'many' 'matter'
 'medium' 'member' 'midst' 'might' 'mountain' 'much' 'muslim' 'must'
 'nat

In [11]:
tfidf = CustomTFIDF()
tfidf_matrix = tfidf.fit_transform(all_unique_words)
print(tfidf_matrix)

[[0.         0.         0.11097037 0.         0.11097037 0.11097037
  0.         0.         0.11097037 0.11097037 0.11097037 0.11097037
  0.08439582 0.         0.         0.         0.         0.
  0.11097037 0.11097037 0.11097037 0.11097037 0.11097037 0.11097037
  0.11097037 0.11097037 0.11097037 0.         0.11097037 0.11097037
  0.11097037 0.         0.         0.11097037 0.11097037 0.
  0.11097037 0.         0.         0.         0.         0.
  0.         0.11097037 0.         0.11097037 0.11097037 0.
  0.         0.         0.11097037 0.         0.11097037 0.11097037
  0.         0.11097037 0.         0.         0.         0.
  0.         0.11097037 0.         0.         0.08439582 0.11097037
  0.11097037 0.11097037 0.         0.         0.11097037 0.11097037
  0.         0.         0.         0.11097037 0.         0.
  0.11097037 0.         0.11097037 0.         0.11097037 0.11097037
  0.11097037 0.08439582 0.         0.11097037 0.         0.11097037
  0.08439582 0.11097037 0.  